# Comparison of optimizers
---

### Comparison of the effectiveness of different optimizers

***Author: Piotr Skalski***

## Imports

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Settings

In [2]:
# starting point for gradient descent
INIT_PARAMS = [1, -0.001]
# output directory (the folder must be created on the drive)
OUTPUT_DIR = "optimizers_comparison"

## Optimisation function

In [3]:
def optimize(tf_function, init_point, iterations, optimizer):
    x, y = [tf.Variable(initial_value=p, dtype=tf.float32) for p in init_point]
    function = tf_function(x, y)
    train_op = optimizer.minimize(function)
    x_list, y_list, cost_list = [], [], []
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for t in range(iterations):
            x_, y_, function_ = sess.run([x, y, function])
            x_list.append(x_); y_list.append(y_); cost_list.append(function_)
            result, _ = sess.run([function, train_op])
            
    return x_list, y_list, cost_list

## Create a blank chart

In [4]:
def create_blank_chart_with_styling(plot_size):
    # my favorite styling kit
    plt.style.use('dark_background')
    # determining the size of the graph
    fig = plt.figure(figsize=plot_size)    
    # 3D mode
    ax = Axes3D(fig)
    # transparent axis pane background 
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    # setting chart axis names
    ax.set(xlabel="$x$", ylabel="$y$")
    return (fig, ax)

## Create animation

In [5]:
# Additional settings
ITERATIONS = 180
GRID_X_MIN = -4
GRID_X_MAX = 4
GRID_Y_MIN = -4
GRID_Y_MAX = 4
LR = 0.02

In [6]:
# Definition of optimisers
optimizers = [
    (tf.train.GradientDescentOptimizer(learning_rate=LR), "Gradient Descent"),
    (tf.train.MomentumOptimizer(learning_rate=LR, momentum=0.9, use_nesterov=False), "Momentum"),
    (tf.train.MomentumOptimizer(learning_rate=LR, momentum=0.9, use_nesterov=True), "Nasterov"),
    (tf.train.RMSPropOptimizer(learning_rate=LR), "RMSProp"),
    (tf.train.AdamOptimizer(learning_rate=LR), "Adam"),
]

In [7]:
# Definition of colours for subsequent trajectories
paths_colors = [
    "#F2112D",
    "#F06E1E",
    "#EED82A",
    "#A5EC37",
    "#54EA43"
]

In [9]:
# Definition of the optimised function
tf_fun = lambda x, y: x**2 - y**2
np_fun = lambda x, y: x**2 - y**2


In [10]:
# Trajectories covered by optimizers
optimization_paths = [optimize(tf_fun, INIT_PARAMS, ITERATIONS, optimizer[0]) for optimizer in optimizers]
labels = [item[1] for item in optimizers]

In [13]:
def create_animation(np_function, iterations, paths, colors, plot_name, file_name, dir_name):
    for angle in range(iterations):
        fix, ax = create_blank_chart_with_styling((10, 10))
        
        a3D, b3D = np.meshgrid(np.linspace(-1, 1, 50), np.linspace(-1, 1, 50))
        cost3D = np.array([np_function(x_, y_) for x_, y_ in zip(a3D.flatten(), b3D.flatten())]).reshape(a3D.shape)
        ax.plot_wireframe(a3D, b3D, cost3D, cmap=plt.get_cmap('rainbow'), alpha=0.2, zorder=-10)
        
        for path, color in zip(paths, colors):
            ax.plot(path[0][:angle], path[1][:angle], zs=path[2][:angle], zdir='z', c=color, lw=3, zorder=1, alpha=1.0)
            
            if angle == 0:
                ax.scatter(path[0][0], path[1][0], zs=path[2][0], s=100, c=color, zorder=10, edgecolors="k")
            else:
                ax.scatter(path[0][angle-1], path[1][angle-1], zs=path[2][angle-1], s=100, c=color, zorder=10, edgecolors="k")
        
        ax.legend(labels, loc='lower right', prop={'size': 10}, framealpha=0.0)
        
        ax.set_xlim(-1, 1)
        ax.set_ylim(-1, 1)
        ax.set_zlim(cost3D.min(), cost3D.max())
        
        # graph rotation
        ax.view_init(45, 180 + angle*2)
        # addition of a title
        ax.set_title(plot_name, fontsize=20)
        # saving a file
        plt.savefig("./{}/{}_{:05}.png".format(dir_name, file_name, angle))
        plt.close()

In [14]:
create_animation(np_fun, ITERATIONS, optimization_paths, paths_colors, "", "test", OUTPUT_DIR)